In [1]:
# For colab
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/Github/KT_Devchall"

Mounted at /content/Mydrive
/content/Mydrive/MyDrive/Github/KT_Devchall


# Preprocessing

In [2]:
import random
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

set_config(display='diagram')

In [4]:
# data load
df_train = pd.read_pickle('data/train.pkl')
X = df_train.drop('Class', axis=1)
y = df_train['Class']

In [5]:
## Column Selection from catboost
feature_importance = pd.Series([3.12768611e+01, 6.62209523e+00, 1.55948023e+01, 1.05818328e+00,
       5.35072461e-02, 0.00000000e+00, 3.21271373e+00, 2.57594949e-02,
       1.17493059e+01, 9.91394069e-02, 9.48666051e+00, 9.53121732e-01,
       5.07253124e+00, 8.31159974e+00, 5.98996443e-02, 3.63612996e+00,
       2.78768950e+00], index = X.columns)
col_drop = feature_importance[feature_importance<0.1] 
col_drop

Platform      0.053507
OS_type       0.000000
Ex_Rate       0.025759
Country_ID    0.099139
weekend       0.059900
dtype: float64

In [6]:
X = X[X.columns.drop(col_drop.index)]

In [7]:
# num_features = ['Ex_Rate','P1','P2','P3','P4']
num_features = ['P1','P2','P3','P4']
cat_features = X.columns.drop(num_features).tolist()

In [8]:
# Transformer
numeric_transformer = StandardScaler()
cat_transformer = OneHotEncoder(drop='first', sparse=False)

In [9]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", cat_transformer, cat_features)
    ]
)

In [12]:
# Train/Val/Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y)

In [14]:
# Fit
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.fit_transform(X_val)

# Sklearn Modeling

In [23]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [24]:
sgd = SGDClassifier(n_jobs=-1, verbose=5, random_state=123, 
                      early_stopping=True, n_iter_no_change=7, max_iter=100,
                      learning_rate = 'adaptive')

In [25]:
# gridsearch hyperparameters
param_grid = { 
    'loss' : ['log_loss', 'hinge', 'modified_huber'],
    'penalty' : ['l2', 'l1'],
    'eta_0' : [0.1, 0.01]
}

In [27]:
model = GridSearchCV(sgd, param_grid, verbose=3, scoring='f1', n_jobs=-1, cv=5)

## GridSearch Fitting

In [28]:
# Fit
model.fit()

TypeError: ignored